In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re

In [38]:
""" 네이버 랭킹 뉴스 긁어오기 """
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'}
d_list = []
start_data = 20220201
end_data = 20220425
for date_int in range(start_data, end_data):
    date = str(date_int)
    url = "https://news.naver.com/main/ranking/popularDay.nhn?date=" + date
    html = requests.get(url, headers=headers).text
    soup = BS(html, 'html.parser')
    ranking_total = soup.find_all(class_='rankingnews_box')

    for item in ranking_total:
        media = item.a.strong.text
        news = item.find_all(class_="list_content")
        
        for new in news:
            d = {}
            d['media'] = media
            d['src'] = "https://news.naver.com/" + new.a['href']
            d['title'] = new.a.text
            d['date'] = date
            d_list.append(d)
df = pd.DataFrame(d_list)

In [41]:
df

,media,src,title,date,title_c,keyword
0,노컷뉴스,https://news.naver.com/https://n.news.naver.co...,코스피 뒤집어 놓은 'LG엔솔 쪼개기 상장' 후유증,20220201,코스피 뒤집어 놓은 LG엔솔 쪼개기 상장 후유증,NaN
1,노컷뉴스,https://news.naver.com/https://n.news.naver.co...,"이준석 ""대장동 자료 거부하는 쪽이 범인…토론 무산 유감""",20220201,이준석 대장동 자료 거부하는 쪽이 범인토론 무산 유감,NaN
2,노컷뉴스,https://news.naver.com/https://n.news.naver.co...,"[영상]""왜 안 잘라줘"" 끓는 기름에 호떡 던진 그 남자…쇠고랑 찼다",20220201,영상왜 안 잘라줘 끓는 기름에 호떡 던진 그 남자쇠고랑 찼다,NaN
3,노컷뉴스,https://news.naver.com/https://n.news.naver.co...,"""몇 대 몇!"" 울림 여전한데…韓방송사 거목 허참의 유훈",20220201,몇 대 몇 울림 여전한데韓방송사 거목 허참의 유훈,NaN
4,노컷뉴스,https://news.naver.com/https://n.news.naver.co...,"""귀요미, 예뻐하는 거 알지?""…제자에게 만남 요구한 교수",20220201,귀요미 예뻐하는 거 알지제자에게 만남 요구한 교수,NaN
...,...,...,...,...,...,...
89214,레이디경향,https://news.naver.com/https://n.news.naver.co...,"[주말&]이맘때면 몸이 기억하는 소풍 김밥의 맛, 4색 레시피",20220424,주말이맘때면 몸이 기억하는 소풍 김밥의 맛 4색 레시피,NaN
89215,레이디경향,https://news.naver.com/https://n.news.naver.co...,대법원 ‘군대 동성 간 성관계’ 무죄 판결…국제엠네스티 “LGBTI 인권에 중요한 승리”,20220424,대법원 군대 동성 간 성관계 무죄 판결국제엠네스티 LGBTI 인권에 중요한 승리,NaN
89216,레이디경향,https://news.naver.com/https://n.news.naver.co...,나이키 모델된 '운동뚱' 김민경의 메시지,20220424,나이키 모델된 운동뚱 김민경의 메시지,NaN
89217,레이디경향,https://news.naver.com/https://n.news.naver.co...,금쪽이들 모여라! 오은영 박사 '1도의 변화' 티켓 오픈,20220424,금쪽이들 모여라 오은영 박사 1도의 변화 티켓 오픈,NaN


In [39]:
""" 필요 없는 문자 제거 """
def clean_text(row):
    text = row['title']
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("E-mail제거 : " , text , "\n")
   
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("URL 제거 : ", text , "\n")
    
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("한글 자음 모음 제거 : ", text , "\n")
    
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("태그 제거 : " , text , "\n")
    
    pattern = r'\([^)]*\)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("괄호와 괄호안 글자 제거 :  " , text , "\n")
    
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("특수기호 제거 : ", text , "\n" )
    
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("필요없는 정보 제거 : ", text , "\n" )
    
    pattern = '["단독"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '["속보"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("단독 속보 제거 : ", text , "\n" )
    
    text = text.strip()
    # print("양 끝 공백 제거 : ", text , "\n" )
    
    text = " ".join(text.split())
    # print("중간에 공백은 1개만 : ", text )
    
    return text

df['title_c'] = df.apply(clean_text, axis=1)

In [40]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran

kkma = Kkma()
komoran = Komoran()
df.loc[idx_line, 'keyword'] =0
df.reset_index(drop=True, inplace =True)
lis = []
for idx_line in range(len(df)):  
    nouns_list = komoran.nouns(df['title_c'].loc[idx_line])
    nouns_list_c = [nouns for nouns in nouns_list if len(nouns) > 1]    # 한글자는 이상한게 많아서 2글자 이상
    lis.append(nouns_list_c)
    # print(nouns_list_c)
    # df.loc[idx_line, 'keyword'] = [list(set(nouns_list_c))]
    
# # df = df[df['media'] != '코리아헤럴드']    # 코리아헤럴드는 영어 제목임
# # df = df[df['media'] != '주간경향']    # 주간경향은 같은 title이 많음

In [ ]:
for idx_line in range(len(df)): 
    nouns_list_c = [nouns for nouns in nouns_list if len(nouns) > 1]  
    

In [54]:
from neo4j import GraphDatabase


""" make node & relationship"""
def add_article(tx, title, date, media, keyword):
    tx.run("MERGE (a:Article {title: $title , date: $date, media: $media, keyword: $keyword})",
           title=title, date=date, media=media, keyword=keyword)


def add_media(tx):
    tx.run("MATCH (a:Article) "
           "MERGE (b:Media {name:a.media}) "
           "MERGE (a)<-[r:Publish]-(b)")


def add_keyword(tx):
    tx.run("MATCH (a:Article) "
           "UNWIND a.keyword as k "
           "MERGE (b:Keyword {name:k}) "
           "MERGE (a)-[r:Include]->(b)")



""" 한자와 공백 제거 """
# Neo4j -> Gephi 에서 parsing error의 원인이 될 수 있음
def clean_text_for_neo4j(row):
    text = row['title_c']
    text = re.sub(pattern='[^a-zA-Z0-9ㄱ-ㅣ가-힣]', repl='', string=text)
    # print("영어, 숫자, 한글만 포함 : ", text )
    return text

df['title_c_neo4j'] = df.apply(clean_text_for_neo4j, axis=1)



""" 연결 """
# Neo4j 브라우저에서 설정한 계정의 ID, PASSWORD를 통해 접속
greeter = GraphDatabase.driver("bolt://localhost:7687", auth=("mastav7@gmail.com", "Ab2041811"))  



""" 입력 """
# Cyper code를 이용,  크롤링한 Data를 DB에 입력
with greeter.session() as session:
    """ make node """
    for idx in range(len(df)):
        session.write_transaction(add_article, title=df.iloc[idx]['title_c_neo4j'], date=df.iloc[idx]['date'],
                                  media=df.iloc[idx]['media'], keyword=list(df.iloc[idx]['keyword']))
    session.write_transaction(add_media)
    session.write_transaction(add_keyword)

Transaction failed and will be retried in 0.8475263716268245s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다))
Transaction failed and will be retried in 1.900035016659824s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다))
Transaction failed and will be retried in 3.3801133318976717s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)

In [43]:
pd.Series(lis)

0                             [코스, 상장, 후유증]
1        [이준석, 대장동, 자료, 거부, 범인, 토론, 무산, 유감]
2                     [영상, 기름, 호떡, 남자, 쇠고랑]
3                     [울림, 방송사, 거목, 허참, 유훈]
4                      [요미, 제자, 만남, 요구, 교수]
                        ...                
89214             [주말, 이맘때, 기억, 소풍, 김밥, 레시]
89215         [대법원, 군대, 동성, 관계, 무죄, 인권, 승리]
89216                   [나이키, 모델, 김민경, 메시지]
89217         [금쪽, 오은영, 박사, 도의, 변화, 티켓, 오픈]
89218                          [리사, 리사, 리사]
Length: 89219, dtype: object